## ETL : YELP - User
Este archivo solo carga informacion de la tabla user.
Como pre requisito es necesario:
+ tener la tabla yelp_business_florida, que contiene los establecimientos del estado FLORIDA
+ tener la tabla yelp_review que contiene la informacion de reviews de establec. del estado FLORIDA

Haciendo un cruce de esas dos tablas sabemos que registros de la tabla User debemos considerar en la carga
para el analisis del estado FLORIDA.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cargamos los reviews de florida
import pandas as pd
df_yelp_review = pd.read_csv("/content/drive/MyDrive/Otros/yelp_review_florida.csv")

In [ ]:
# YELP : carga de informacion user
import pyarrow.parquet as pq
import pandas as pd
df_yelp_user=pd.DataFrame()
parquet_file = pq.ParquetFile("/content/drive/MyDrive/yelp/Copia de user.parquet")
for i in parquet_file.iter_batches(batch_size=10000):
    df_chunk = i.to_pandas()
    df_yelp_user = pd.concat([df_yelp_user, df_chunk], ignore_index=True)

In [ ]:
df_yelp_user = df_yelp_user[df_yelp_user['user_id'].isin(df_yelp_review['user_id'])]

In [ ]:
df_yelp_user['friends_count'] = df_yelp_user['friends'].apply(lambda x: len(x.split(',')))
df_yelp_user['elite_count'] = df_yelp_user['elite'].apply(lambda x: len(x.split(',')))


In [ ]:
df_yelp_user = df_yelp_user.drop(['elite','friends'], axis=1)

In [ ]:
df_yelp_user.to_csv('/content/drive/MyDrive/Otros/yelp_user.csv', index=False)